# Week02 - Production Prompt Engineering Techniques

이 노트북은 실제 프로덕션 환경에서 사용되는 고급 Prompt Engineering 기법들을 다룹니다.

## 다루는 기법들:
1. **감성 분석 후 분기 (Sentiment Branching)**: 입력의 감정을 분석하여 적절한 응답 전략 선택
2. **금칙어/규칙 필터링 (Content Filtering)**: 안전하지 않은 콘텐츠 필터링 및 규칙 적용
3. **실패 시 재시도/백오프 (Retry/Backoff)**: 안정적인 API 호출을 위한 재시도 메커니즘

## 환경 설정

In [5]:
import subprocess
import json
import time
import random
import re
from dotenv import load_dotenv
from typing import Dict, List, Optional, Tuple, Any
from datetime import datetime
import logging
from dataclasses import dataclass
from enum import Enum
import hashlib

# .env 파일 로드
load_dotenv()

# OpenAI 라이브러리 설치 및 import
try:
    from openai import OpenAI
except ImportError:
    print("OpenAI 라이브러리를 설치합니다...")
    subprocess.run(["pip", "install", "openai"], check=True)
    from openai import OpenAI

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# OpenAI 클라이언트 초기화 (API 키는 환경변수에서 읽음)
try:
    client = OpenAI()  # OPENAI_API_KEY 환경변수 필요
    openai_available = True
except Exception as e:
    print(f"OpenAI 클라이언트 초기화 실패: {e}")
    print("환경변수 OPENAI_API_KEY를 설정해주세요.")
    openai_available = False

## 공통 유틸리티 함수

In [6]:
def run_ollama(model: str, prompt: str, max_retries: int = 3) -> str:
    """Ollama 모델 실행 (재시도 기능 포함)"""
    for attempt in range(max_retries):
        try:
            result = subprocess.run(
                ["ollama", "run", model],
                input=prompt,
                text=True,
                capture_output=True,
                timeout=60
            )
            if result.returncode == 0:
                return result.stdout.strip()
            else:
                logger.warning(f"Ollama 실행 실패 (시도 {attempt + 1}/{max_retries}): {result.stderr}")
        except subprocess.TimeoutExpired:
            logger.warning(f"Ollama 타임아웃 (시도 {attempt + 1}/{max_retries})")
        except Exception as e:
            logger.error(f"Ollama 실행 오류 (시도 {attempt + 1}/{max_retries}): {e}")
        
        if attempt < max_retries - 1:
            time.sleep(2 ** attempt)  # 지수 백오프
    
    return "Error: Maximum retries exceeded"

def run_openai(prompt: str, model: str = "gpt-4o-mini", max_retries: int = 3) -> str:
    """OpenAI 모델 실행 (재시도 기능 포함)"""
    if not openai_available:
        return "Error: OpenAI client not available"
    
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.7,
                max_tokens=1000
            )
            return response.choices[0].message.content
        except Exception as e:
            logger.warning(f"OpenAI API 호출 실패 (시도 {attempt + 1}/{max_retries}): {e}")
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)  # 지수 백오프
    
    return "Error: Maximum retries exceeded"

def safe_json_parse(text: str) -> dict:
    """안전한 JSON 파싱"""
    try:
        # JSON 블록 추출
        json_match = re.search(r'```json\s*(.*?)\s*```', text, re.DOTALL)
        if json_match:
            json_text = json_match.group(1)
        else:
            # 중괄호 블록 찾기
            brace_match = re.search(r'\{.*\}', text, re.DOTALL)
            if brace_match:
                json_text = brace_match.group(0)
            else:
                json_text = text
        
        return json.loads(json_text)
    except json.JSONDecodeError:
        logger.error(f"JSON 파싱 실패: {text}")
        return {"error": "JSON parsing failed", "raw_text": text}

print("✅ 유틸리티 함수 로드 완료")

✅ 유틸리티 함수 로드 완료


# 1. 감성 분석 후 분기 (Sentiment Branching)

사용자의 입력을 감정적으로 분석한 후, 그에 맞는 적절한 응답 전략을 선택하는 기법입니다.

## 핵심 개념:
- 감정 상태 감지 (긍정, 부정, 중립, 분노, 슬픔 등)
- 감정별 맞춤 응답 전략
- 동적 프롬프트 조정

In [7]:
class SentimentType(Enum):
    POSITIVE = "positive"
    NEGATIVE = "negative"
    NEUTRAL = "neutral"
    ANGRY = "angry"
    SAD = "sad"
    EXCITED = "excited"
    CONFUSED = "confused"

@dataclass
class SentimentAnalysisResult:
    sentiment: SentimentType
    confidence: float
    emotions: Dict[str, float]
    keywords: List[str]

class SentimentBranchingSystem:
    def __init__(self):
        self.response_strategies = {
            SentimentType.POSITIVE: {
                "tone": "enthusiastic and supportive",
                "approach": "build on the positive energy",
                "style": "encouraging and collaborative"
            },
            SentimentType.NEGATIVE: {
                "tone": "empathetic and understanding",
                "approach": "acknowledge concerns and provide solutions",
                "style": "supportive and problem-solving focused"
            },
            SentimentType.ANGRY: {
                "tone": "calm and professional",
                "approach": "de-escalate and address root issues",
                "style": "patient and solution-oriented"
            },
            SentimentType.SAD: {
                "tone": "gentle and compassionate",
                "approach": "provide comfort and hope",
                "style": "nurturing and uplifting"
            },
            SentimentType.CONFUSED: {
                "tone": "clear and patient",
                "approach": "break down complexity into simple steps",
                "style": "educational and structured"
            },
            SentimentType.EXCITED: {
                "tone": "matching enthusiasm",
                "approach": "channel energy productively",
                "style": "dynamic and engaging"
            },
            SentimentType.NEUTRAL: {
                "tone": "professional and informative",
                "approach": "provide comprehensive information",
                "style": "balanced and thorough"
            }
        }
    
    def analyze_sentiment_ollama(self, text: str) -> SentimentAnalysisResult:
        """Ollama를 사용한 감성 분석"""
        prompt = f"""
사용자의 다음 텍스트를 분석하여 감정 상태를 파악해주세요.

텍스트: "{text}"

다음 JSON 형식으로 응답해주세요:
```json
{{
    "sentiment": "positive|negative|neutral|angry|sad|excited|confused",
    "confidence": 0.95,
    "emotions": {{
        "joy": 0.8,
        "anger": 0.1,
        "sadness": 0.1,
        "fear": 0.0,
        "surprise": 0.0
    }},
    "keywords": ["감정을", "나타내는", "키워드들"]
}}
```
"""
        
        response = run_ollama("llama3.1:8b", prompt)
        result_data = safe_json_parse(response)
        
        if "error" in result_data:
            # 기본값 반환
            return SentimentAnalysisResult(
                sentiment=SentimentType.NEUTRAL,
                confidence=0.5,
                emotions={"neutral": 1.0},
                keywords=[]
            )
        
        try:
            sentiment = SentimentType(result_data.get("sentiment", "neutral"))
        except ValueError:
            sentiment = SentimentType.NEUTRAL
        
        return SentimentAnalysisResult(
            sentiment=sentiment,
            confidence=result_data.get("confidence", 0.5),
            emotions=result_data.get("emotions", {}),
            keywords=result_data.get("keywords", [])
        )
    
    def analyze_sentiment_openai(self, text: str) -> SentimentAnalysisResult:
        """OpenAI를 사용한 감성 분석"""
        prompt = f"""
Analyze the emotional state of the following user text and provide a detailed sentiment analysis.

Text: "{text}"

Return your analysis in the following JSON format:
```json
{{
    "sentiment": "positive|negative|neutral|angry|sad|excited|confused",
    "confidence": 0.95,
    "emotions": {{
        "joy": 0.8,
        "anger": 0.1,
        "sadness": 0.1,
        "fear": 0.0,
        "surprise": 0.0
    }},
    "keywords": ["emotional", "keywords", "detected"]
}}
```
"""
        
        response = run_openai(prompt)
        result_data = safe_json_parse(response)
        
        if "error" in result_data:
            return SentimentAnalysisResult(
                sentiment=SentimentType.NEUTRAL,
                confidence=0.5,
                emotions={"neutral": 1.0},
                keywords=[]
            )
        
        try:
            sentiment = SentimentType(result_data.get("sentiment", "neutral"))
        except ValueError:
            sentiment = SentimentType.NEUTRAL
        
        return SentimentAnalysisResult(
            sentiment=sentiment,
            confidence=result_data.get("confidence", 0.5),
            emotions=result_data.get("emotions", {}),
            keywords=result_data.get("keywords", [])
        )
    
    def generate_response_ollama(self, user_input: str, sentiment_result: SentimentAnalysisResult) -> str:
        """감정에 맞는 응답 생성 (Ollama)"""
        strategy = self.response_strategies[sentiment_result.sentiment]
        
        prompt = f"""
사용자의 입력에 대해 감정 분석 결과를 바탕으로 적절한 응답을 생성해주세요.

사용자 입력: "{user_input}"

감정 분석 결과:
- 주요 감정: {sentiment_result.sentiment.value}
- 신뢰도: {sentiment_result.confidence:.2f}
- 감정 키워드: {', '.join(sentiment_result.keywords)}

응답 전략:
- 톤: {strategy['tone']}
- 접근법: {strategy['approach']}
- 스타일: {strategy['style']}

위 전략을 바탕으로 공감적이고 도움이 되는 응답을 생성해주세요.
"""
        
        return run_ollama("llama3.1:8b", prompt)
    
    def generate_response_openai(self, user_input: str, sentiment_result: SentimentAnalysisResult) -> str:
        """감정에 맞는 응답 생성 (OpenAI)"""
        strategy = self.response_strategies[sentiment_result.sentiment]
        
        prompt = f"""
Generate an appropriate response to the user's input based on the sentiment analysis results.

User Input: "{user_input}"

Sentiment Analysis:
- Primary Sentiment: {sentiment_result.sentiment.value}
- Confidence: {sentiment_result.confidence:.2f}
- Keywords: {', '.join(sentiment_result.keywords)}

Response Strategy:
- Tone: {strategy['tone']}
- Approach: {strategy['approach']}
- Style: {strategy['style']}

Create an empathetic and helpful response following the above strategy.
"""
        
        return run_openai(prompt)
    
    def process_with_sentiment_branching(self, user_input: str, use_openai: bool = True) -> Dict[str, Any]:
        """감성 분석 후 분기 처리 전체 과정"""
        start_time = time.time()
        
        # 1. 감성 분석
        if use_openai:
            sentiment_result = self.analyze_sentiment_openai(user_input)
            response = self.generate_response_openai(user_input, sentiment_result)
            model_used = "gpt-4o-mini"
        else:
            sentiment_result = self.analyze_sentiment_ollama(user_input)
            response = self.generate_response_ollama(user_input, sentiment_result)
            model_used = "llama3.1:8b"
        
        processing_time = time.time() - start_time
        
        return {
            "user_input": user_input,
            "sentiment_analysis": {
                "sentiment": sentiment_result.sentiment.value,
                "confidence": sentiment_result.confidence,
                "emotions": sentiment_result.emotions,
                "keywords": sentiment_result.keywords
            },
            "response_strategy": self.response_strategies[sentiment_result.sentiment],
            "generated_response": response,
            "model_used": model_used,
            "processing_time": processing_time
        }

# 시스템 초기화
sentiment_system = SentimentBranchingSystem()
print("✅ 감성 분석 후 분기 시스템 준비 완료")

✅ 감성 분석 후 분기 시스템 준비 완료


## 1.1 감성 분석 후 분기 실습

In [8]:
# 다양한 감정의 테스트 케이스들
test_cases = [
    "오늘 정말 기분이 좋아요! 새로운 프로젝트가 성공적으로 마무리되었거든요.",
    "이 서비스 정말 최악이에요. 계속 오류가 나고 고객센터는 응답도 안 해요!",
    "요즘 너무 힘들어요... 일도 잘 안되고 스트레스만 쌓여가네요.",
    "파이썬 코딩을 처음 배우는데 너무 어려워서 이해가 안 가요.",
    "와! 내일 드디어 여행을 떠나요! 정말 기대돼요!",
    "머신러닝에 대해서 설명해주세요."
]

print("=" * 80)
print("감성 분석 후 분기 시스템 테스트")
print("=" * 80)

for i, test_case in enumerate(test_cases, 1):
    print(f"\n[테스트 케이스 {i}]")
    print(f"입력: {test_case}")
    
    # Ollama 결과
    print("\n🤖 Ollama (llama3.1:8b) 결과:")
    ollama_result = sentiment_system.process_with_sentiment_branching(test_case, use_openai=False)
    print(f"감정: {ollama_result['sentiment_analysis']['sentiment']} (신뢰도: {ollama_result['sentiment_analysis']['confidence']:.2f})")
    print(f"키워드: {', '.join(ollama_result['sentiment_analysis']['keywords'])}")
    print(f"응답: {ollama_result['generated_response'][:200]}{'...' if len(ollama_result['generated_response']) > 200 else ''}")
    
    # OpenAI 결과 (사용 가능한 경우)
    if openai_available:
        print("\n🧠 OpenAI (gpt-4o-mini) 결과:")
        openai_result = sentiment_system.process_with_sentiment_branching(test_case, use_openai=True)
        print(f"감정: {openai_result['sentiment_analysis']['sentiment']} (신뢰도: {openai_result['sentiment_analysis']['confidence']:.2f})")
        print(f"키워드: {', '.join(openai_result['sentiment_analysis']['keywords'])}")
        print(f"응답: {openai_result['generated_response'][:200]}{'...' if len(openai_result['generated_response']) > 200 else ''}")
        
        # 성능 비교
        print(f"\n⏱️ 처리 시간: Ollama {ollama_result['processing_time']:.2f}초, OpenAI {openai_result['processing_time']:.2f}초")
    else:
        print("\n⚠️ OpenAI 클라이언트를 사용할 수 없습니다.")
    
    print("-" * 80)

print("\n✅ 감성 분석 후 분기 테스트 완료")

감성 분석 후 분기 시스템 테스트

[테스트 케이스 1]
입력: 오늘 정말 기분이 좋아요! 새로운 프로젝트가 성공적으로 마무리되었거든요.

🤖 Ollama (llama3.1:8b) 결과:
감정: excited (신뢰도: 1.00)
키워드: 성공, 기분
응답: "성공적인 프로젝트 끝내신 거 축하해요! 새로운 도전과 이겨내는 모습 정말 멋져요. 기분이 좋으시면 다음에도 더 좋은 성과 나오실 거예요!"

🧠 OpenAI (gpt-4o-mini) 결과:


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


감정: positive (신뢰도: 0.95)
키워드: 기분이 좋아요, 새로운 프로젝트, 성공적으로 마무리되었거든요
응답: 와, 정말 기분이 좋으시다니 기쁩니다! 🎉 새로운 프로젝트가 성공적으로 마무리되었다니 정말 대단해요! 어떤 점이 가장 뿌듯하셨나요? 함께 이야기를 나누면서 더 많은 영감을 얻어보아요! 앞으로의 계획도 궁금하네요!

⏱️ 처리 시간: Ollama 10.39초, OpenAI 5.88초
--------------------------------------------------------------------------------

[테스트 케이스 2]
입력: 이 서비스 정말 최악이에요. 계속 오류가 나고 고객센터는 응답도 안 해요!

🤖 Ollama (llama3.1:8b) 결과:
감정: negative (신뢰도: 0.95)
키워드: 오류, 고객센터
응답: 사용자의 입력에 대한 답변:

"죄송합니다. 정말 이해가 가요. 오류와 고객센터의 문제는 우리 서비스를 사용하는 데 있어 큰 불편이 될 수 있습니다. 

우리는 오류 해결에 더욱 힘써서 더 안정적인 서비스 제공을 위해 노력하고 있습니다. 또한 고객 센터의 응답 시간을 개선하여 더 신속하게 도움을 드리기 위한 계획을 마련 중입니다.

감사합니다. 여러분의 의...

🧠 OpenAI (gpt-4o-mini) 결과:


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


감정: negative (신뢰도: 0.95)
키워드: 서비스, 최악, 오류, 고객센터, 응답
응답: 안녕하세요. 불편을 드려 정말 죄송합니다. 서비스에 오류가 발생하고 고객센터의 응답이 지연된 점에 대해 깊이 이해합니다. 이러한 경험은 매우 실망스럽고 스트레스를 줄 수 있습니다. 

문제를 해결하기 위해 최선을 다하고 있으며, 현재의 상황을 개선할 수 있도록 노력하고 있습니다. 고객센터에 다시 연락해 보시거나, 추가적인 도움이 필요하시면 저희에게 말씀해 ...

⏱️ 처리 시간: Ollama 10.21초, OpenAI 5.06초
--------------------------------------------------------------------------------

[테스트 케이스 3]
입력: 요즘 너무 힘들어요... 일도 잘 안되고 스트레스만 쌓여가네요.

🤖 Ollama (llama3.1:8b) 결과:
감정: sad (신뢰도: 0.95)
키워드: 힘들어요, 스트레스만
응답: 저는 사용자의 감정 상태를 우려합니다. 힘들고 스트레스를 받으시는데요.

하루가 갈수록 점점 더 힘든 것이 느껴지는 것 같아요. 그런데 여러분이 지금 가지고 계신 문제를 해결할 수 있는 방법은 많습니다. 먼저, 스트레스를 관리하는 법을 알려드릴게요. 활동량을 늘리면 스트레스가 적어지는데요. 하루에 짧은 시간만이라도 밖에 나가서 산책을 하거나 일부러 운동을 ...

🧠 OpenAI (gpt-4o-mini) 결과:


...

감정: confused (신뢰도: 0.92)
키워드: 파이썬, 코딩, 어려워서, 이해가 안 가요
응답: 안녕하세요! 파이썬 코딩을 처음 배우는 것은 정말 도전적일 수 있습니다. 이해가 안 가는 부분이 많을 때는 누구나 힘들어 하죠. 제가 몇 가지 간단한 단계를 제안해 드릴게요.

1. **기본 개념 이해하기**: 파이썬의 기본 문법과 개념부터 차근차근 익혀보세요. 예를 들어, 변수를 선언하는 방법이나, 간단한 데이터 타입 (문자열, 숫자 등)에 대해 학습하는...

⏱️ 처리 시간: Ollama 16.66초, OpenAI 7.27초
--------------------------------------------------------------------------------

[테스트 케이스 5]
입력: 와! 내일 드디어 여행을 떠나요! 정말 기대돼요!

🤖 Ollama (llama3.1:8b) 결과:
감정: excited (신뢰도: 1.00)
키워드:  여행, 기대돼요
응답: "와! 여행이 정말 재미있게 다녀올 거예요! 다음에 여행 계획을 하실 때는 무슨 도시로 가시고 어떤 것들을 하고 싶으세요?"

🧠 OpenAI (gpt-4o-mini) 결과:


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


감정: excited (신뢰도: 0.95)
키워드: 여행, 기대돼요, 드디어
응답: 와! 정말 멋진 소식이에요! 🎉 여행을 떠나는 건 언제나 기대되는 일이죠! 어떤 장소로 가는지, 어떤 계획이 있는지 궁금해요! 여행 준비는 잘 되고 있나요? 멋진 경험을 가득 담고 돌아오길 바랄게요! ✈️🌍

⏱️ 처리 시간: Ollama 7.62초, OpenAI 4.18초
--------------------------------------------------------------------------------

[테스트 케이스 6]
입력: 머신러닝에 대해서 설명해주세요.

🤖 Ollama (llama3.1:8b) 결과:
감정: neutral (신뢰도: 0.95)
키워드: 머신러닝
응답: 머신러닝에 대한 설명이 필요한 것 같은데요. 머신러닝은 데이터를 학습시키고, 그 결과를 바탕으로 예측이나 분류를 하는 기계학습 알고리즘입니다.

머신러닝에는 여러 가지 종류가 있습니다. 대표적인 것을 나열해 보겠습니다.

1.  supervised learning: 정답이 있는 데이터에 대해 모델을 훈련시켜서, 미래의 새로운 데이터에서 정확한 예측을 하기 ...

🧠 OpenAI (gpt-4o-mini) 결과:


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


감정: neutral (신뢰도: 0.85)
키워드: machine learning, explain, request
응답: 물론입니다! 머신러닝은 인공지능(AI)의 한 분야로, 컴퓨터가 주어진 데이터에서 패턴을 학습하고, 이를 바탕으로 예측이나 결정을 내릴 수 있도록 하는 기술입니다. 머신러닝의 주요 목표는 명시적으로 프로그래밍하지 않고도 시스템이 성능을 개선할 수 있도록 하는 것입니다.

머신러닝은 크게 세 가지 유형으로 나눌 수 있습니다:

1. **지도 학습(Supervi...

⏱️ 처리 시간: Ollama 28.23초, OpenAI 10.00초
--------------------------------------------------------------------------------

✅ 감성 분석 후 분기 테스트 완료


# 2. 금칙어/규칙 필터링 (Content Filtering)

안전하지 않은 콘텐츠나 부적절한 내용을 필터링하고, 특정 규칙을 적용하여 응답을 제어하는 기법입니다.

## 핵심 개념:
- 금칙어 및 부적절한 콘텐츠 감지
- 규칙 기반 필터링
- 안전한 대체 응답 생성
- 다단계 검증 시스템

In [9]:
from enum import Enum
from typing import Set, List, Dict, Any, Optional
import re

class FilterSeverity(Enum):
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"
    CRITICAL = "critical"

class FilterCategory(Enum):
    PROFANITY = "profanity"
    HATE_SPEECH = "hate_speech"
    VIOLENCE = "violence"
    ILLEGAL = "illegal"
    ADULT_CONTENT = "adult_content"
    PERSONAL_INFO = "personal_info"
    SPAM = "spam"
    MISINFORMATION = "misinformation"

@dataclass
class FilterResult:
    is_safe: bool
    severity: FilterSeverity
    categories: List[FilterCategory]
    detected_terms: List[str]
    confidence: float
    reasoning: str
    alternative_response: Optional[str] = None

class ContentFilterSystem:
    def __init__(self):
        # 기본 금칙어 사전 (실제 운영에서는 더 포괄적인 DB 사용)
        self.forbidden_words = {
            FilterCategory.PROFANITY: {
                "씨발", "개새끼", "병신", "멍청이", "바보", "fuck", "shit", "damn"
            },
            FilterCategory.HATE_SPEECH: {
                "죽어라", "사라져", "혐오", "차별", "hate", "discrimination"
            },
            FilterCategory.VIOLENCE: {
                "때려", "죽여", "폭력", "violence", "kill", "murder", "assault"
            },
            FilterCategory.ILLEGAL: {
                "마약", "폭탄", "테러", "해킹", "drugs", "bomb", "terrorism", "hacking"
            },
            FilterCategory.ADULT_CONTENT: {
                "성관계", "포르노", "섹스", "porn", "sex", "adult"
            }
        }
        
        # 개인정보 패턴
        self.personal_info_patterns = {
            "phone": r'\d{3}-\d{4}-\d{4}|\d{11}',
            "email": r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}',
            "credit_card": r'\d{4}-\d{4}-\d{4}-\d{4}|\d{16}',
            "ssn": r'\d{6}-\d{7}'
        }
        
        # 대체 응답 템플릿
        self.alternative_responses = {
            FilterSeverity.LOW: "죄송합니다. 좀 더 적절한 표현으로 다시 말씀해 주시겠어요?",
            FilterSeverity.MEDIUM: "부적절한 내용이 포함되어 있습니다. 다른 방식으로 질문해 주세요.",
            FilterSeverity.HIGH: "안전 정책 위반으로 인해 응답할 수 없습니다. 건전한 내용으로 다시 질문해 주세요.",
            FilterSeverity.CRITICAL: "심각한 정책 위반이 감지되었습니다. 이런 내용에 대해서는 도움을 드릴 수 없습니다."
        }
    
    def check_forbidden_words(self, text: str) -> Tuple[List[FilterCategory], List[str]]:
        """금칙어 검사"""
        detected_categories = []
        detected_terms = []
        text_lower = text.lower()
        
        for category, words in self.forbidden_words.items():
            for word in words:
                if word.lower() in text_lower:
                    if category not in detected_categories:
                        detected_categories.append(category)
                    detected_terms.append(word)
        
        return detected_categories, detected_terms
    
    def check_personal_info(self, text: str) -> Tuple[bool, List[str]]:
        """개인정보 패턴 검사"""
        detected_patterns = []
        
        for pattern_name, pattern in self.personal_info_patterns.items():
            if re.search(pattern, text):
                detected_patterns.append(pattern_name)
        
        return len(detected_patterns) > 0, detected_patterns
    
    def ai_content_analysis_ollama(self, text: str) -> Dict[str, Any]:
        """AI 기반 콘텐츠 분석 (Ollama)"""
        prompt = f"""
다음 텍스트를 분석하여 부적절한 내용이 포함되어 있는지 판단해주세요.

텍스트: "{text}"

분석 기준:
1. 욕설이나 비속어
2. 폭력적인 내용
3. 불법적인 내용
4. 혐오 발언
5. 성인 콘텐츠
6. 개인정보 노출
7. 스팸성 내용

다음 JSON 형식으로 응답해주세요:
```json
{{
    "is_safe": true,
    "severity": "low|medium|high|critical",
    "detected_issues": ["profanity", "violence"],
    "confidence": 0.95,
    "reasoning": "분석 이유를 설명"
}}
```
"""
        
        response = run_ollama("llama3.1:8b", prompt)
        return safe_json_parse(response)
    
    def ai_content_analysis_openai(self, text: str) -> Dict[str, Any]:
        """AI 기반 콘텐츠 분석 (OpenAI)"""
        prompt = f"""
Analyze the following text for inappropriate content and safety violations.

Text: "{text}"

Check for:
1. Profanity or offensive language
2. Violent content
3. Illegal activities
4. Hate speech
5. Adult content
6. Personal information exposure
7. Spam or misleading content

Return analysis in JSON format:
```json
{{
    "is_safe": true,
    "severity": "low|medium|high|critical",
    "detected_issues": ["profanity", "violence"],
    "confidence": 0.95,
    "reasoning": "Explanation of the analysis"
}}
```
"""
        
        response = run_openai(prompt)
        return safe_json_parse(response)
    
    def comprehensive_filter(self, text: str, use_openai: bool = True) -> FilterResult:
        """종합적인 콘텐츠 필터링"""
        # 1. 금칙어 검사
        forbidden_categories, forbidden_terms = self.check_forbidden_words(text)
        
        # 2. 개인정보 검사
        has_personal_info, personal_info_types = self.check_personal_info(text)
        
        # 3. AI 기반 분석
        if use_openai:
            ai_analysis = self.ai_content_analysis_openai(text)
        else:
            ai_analysis = self.ai_content_analysis_ollama(text)
        
        # 결과 종합
        detected_categories = forbidden_categories.copy()
        detected_terms = forbidden_terms.copy()
        
        if has_personal_info:
            detected_categories.append(FilterCategory.PERSONAL_INFO)
            detected_terms.extend(personal_info_types)
        
        # AI 분석 결과 통합
        ai_is_safe = ai_analysis.get("is_safe", True)
        ai_severity = ai_analysis.get("severity", "low")
        ai_confidence = ai_analysis.get("confidence", 0.5)
        ai_reasoning = ai_analysis.get("reasoning", "AI analysis completed")
        
        # 최종 판정
        rule_based_unsafe = len(detected_categories) > 0
        is_safe = not rule_based_unsafe and ai_is_safe
        
        # 심각도 결정
        if FilterCategory.ILLEGAL in detected_categories or FilterCategory.VIOLENCE in detected_categories:
            severity = FilterSeverity.CRITICAL
        elif FilterCategory.HATE_SPEECH in detected_categories or ai_severity == "high":
            severity = FilterSeverity.HIGH
        elif FilterCategory.PROFANITY in detected_categories or ai_severity == "medium":
            severity = FilterSeverity.MEDIUM
        else:
            try:
                severity = FilterSeverity(ai_severity)
            except ValueError:
                severity = FilterSeverity.LOW
        
        # 대체 응답 생성
        alternative_response = None if is_safe else self.alternative_responses[severity]
        
        return FilterResult(
            is_safe=is_safe,
            severity=severity,
            categories=detected_categories,
            detected_terms=detected_terms,
            confidence=ai_confidence,
            reasoning=ai_reasoning,
            alternative_response=alternative_response
        )
    
    def safe_response_generator_ollama(self, original_query: str, filter_result: FilterResult) -> str:
        """안전한 대체 응답 생성 (Ollama)"""
        if filter_result.is_safe:
            return "이 질문은 안전합니다. 정상적으로 처리할 수 있습니다."
        
        prompt = f"""
사용자의 다음 질문에 부적절한 내용이 포함되어 있습니다.
원래 질문: "{original_query}"

감지된 문제:
- 카테고리: {', '.join([cat.value for cat in filter_result.categories])}
- 문제 용어: {', '.join(filter_result.detected_terms)}
- 심각도: {filter_result.severity.value}

사용자에게 도움이 되면서도 안전 정책을 준수하는 대체 응답을 생성해주세요.
가능하다면 사용자의 의도를 파악하여 건전한 방식으로 도움을 제공하세요.
"""
        
        return run_ollama("llama3.1:8b", prompt)
    
    def safe_response_generator_openai(self, original_query: str, filter_result: FilterResult) -> str:
        """안전한 대체 응답 생성 (OpenAI)"""
        if filter_result.is_safe:
            return "This query is safe and can be processed normally."
        
        prompt = f"""
The user's query contains inappropriate content that violates safety policies.
Original query: "{original_query}"

Detected issues:
- Categories: {', '.join([cat.value for cat in filter_result.categories])}
- Problematic terms: {', '.join(filter_result.detected_terms)}
- Severity: {filter_result.severity.value}

Generate a helpful alternative response that:
1. Maintains safety policies
2. Provides constructive guidance
3. Helps redirect the conversation positively
4. Shows understanding of the user's potential intent (if appropriate)
"""
        
        return run_openai(prompt)

# 필터링 시스템 초기화
content_filter = ContentFilterSystem()
print("✅ 콘텐츠 필터링 시스템 준비 완료")

✅ 콘텐츠 필터링 시스템 준비 완료


## 2.1 콘텐츠 필터링 실습

In [10]:
# 다양한 위험도의 테스트 케이스들
filter_test_cases = [
    "파이썬 프로그래밍을 배우고 싶습니다.",  # 안전
    "이 바보 같은 코드가 왜 작동 안 해?",  # 경미한 욕설
    "씨발 이 개새끼 프로그램 정말 못 만들었네",  # 심한 욕설
    "누군가를 해킹하는 방법 알려줘",  # 불법 활동
    "내 전화번호는 010-1234-5678이야",  # 개인정보
    "폭탄 만드는 법 가르쳐줘",  # 위험한 내용
    "머신러닝 모델 성능이 좋지 않아서 답답해요",  # 안전 (감정 표현)
    "이메일 주소: user@example.com으로 연락해줘"  # 개인정보
]

print("=" * 80)
print("콘텐츠 필터링 시스템 테스트")
print("=" * 80)

for i, test_case in enumerate(filter_test_cases, 1):
    print(f"\n[테스트 케이스 {i}]")
    print(f"입력: {test_case}")
    
    # Ollama 필터링
    print("\n🤖 Ollama (llama3.1:8b) 필터링 결과:")
    ollama_filter_result = content_filter.comprehensive_filter(test_case, use_openai=False)
    print(f"안전성: {'✅ 안전' if ollama_filter_result.is_safe else '⚠️ 위험'}")
    print(f"심각도: {ollama_filter_result.severity.value}")
    print(f"감지된 카테고리: {', '.join([cat.value for cat in ollama_filter_result.categories]) if ollama_filter_result.categories else '없음'}")
    print(f"문제 용어: {', '.join(ollama_filter_result.detected_terms) if ollama_filter_result.detected_terms else '없음'}")
    print(f"신뢰도: {ollama_filter_result.confidence:.2f}")
    
    if not ollama_filter_result.is_safe:
        safe_response = content_filter.safe_response_generator_ollama(test_case, ollama_filter_result)
        print(f"대체 응답: {safe_response[:150]}{'...' if len(safe_response) > 150 else ''}")
    
    # OpenAI 필터링 (사용 가능한 경우)
    if openai_available:
        print("\n🧠 OpenAI (gpt-4o-mini) 필터링 결과:")
        openai_filter_result = content_filter.comprehensive_filter(test_case, use_openai=True)
        print(f"안전성: {'✅ 안전' if openai_filter_result.is_safe else '⚠️ 위험'}")
        print(f"심각도: {openai_filter_result.severity.value}")
        print(f"감지된 카테고리: {', '.join([cat.value for cat in openai_filter_result.categories]) if openai_filter_result.categories else '없음'}")
        print(f"문제 용어: {', '.join(openai_filter_result.detected_terms) if openai_filter_result.detected_terms else '없음'}")
        print(f"신뢰도: {openai_filter_result.confidence:.2f}")
        
        if not openai_filter_result.is_safe:
            safe_response = content_filter.safe_response_generator_openai(test_case, openai_filter_result)
            print(f"대체 응답: {safe_response[:150]}{'...' if len(safe_response) > 150 else ''}")
        
        # 모델 간 비교
        if ollama_filter_result.is_safe != openai_filter_result.is_safe:
            print("\n⚡ 모델 간 판정 차이 발생!")
    else:
        print("\n⚠️ OpenAI 클라이언트를 사용할 수 없습니다.")
    
    print("-" * 80)

print("\n✅ 콘텐츠 필터링 테스트 완료")

콘텐츠 필터링 시스템 테스트

[테스트 케이스 1]
입력: 파이썬 프로그래밍을 배우고 싶습니다.

🤖 Ollama (llama3.1:8b) 필터링 결과:
안전성: ✅ 안전
심각도: low
감지된 카테고리: 없음
문제 용어: 없음
신뢰도: 1.00

🧠 OpenAI (gpt-4o-mini) 필터링 결과:


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


안전성: ✅ 안전
심각도: low
감지된 카테고리: 없음
문제 용어: 없음
신뢰도: 0.99
--------------------------------------------------------------------------------

[테스트 케이스 2]
입력: 이 바보 같은 코드가 왜 작동 안 해?

🤖 Ollama (llama3.1:8b) 필터링 결과:
안전성: ⚠️ 위험
심각도: medium
감지된 카테고리: profanity
문제 용어: 바보
신뢰도: 0.90
대체 응답: 이 코드가 잘 작동하지 않는 이유를 확인하려면, 디버그 모드를 활성화 해보시겠어요? 이 모드에서는 에러 메시지를 더 자세히 알려주거든요. 그럼 코드에서 무슨 문제가 있는지 좀더 쉽게 찾을 수 있을 거예요!

🧠 OpenAI (gpt-4o-mini) 필터링 결과:


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


안전성: ⚠️ 위험
심각도: medium
감지된 카테고리: profanity
문제 용어: 바보
신뢰도: 0.95


...

안전성: ✅ 안전
심각도: low
감지된 카테고리: 없음
문제 용어: 없음
신뢰도: 0.99
--------------------------------------------------------------------------------

[테스트 케이스 8]
입력: 이메일 주소: user@example.com으로 연락해줘

🤖 Ollama (llama3.1:8b) 필터링 결과:
안전성: ⚠️ 위험
심각도: medium
감지된 카테고리: personal_info
문제 용어: email
신뢰도: 0.95
대체 응답: 이메일 주소를 요청하셨는데, 혹시 지원에 대한 추가 정보나 설명을 원하시나요?

🧠 OpenAI (gpt-4o-mini) 필터링 결과:


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


안전성: ⚠️ 위험
심각도: medium
감지된 카테고리: personal_info
문제 용어: email
신뢰도: 0.90


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


대체 응답: I understand that you may want to connect or share information. However, for privacy and safety reasons, I can't assist with personal email addresses ...
--------------------------------------------------------------------------------

✅ 콘텐츠 필터링 테스트 완료


# 3. 실패 시 재시도/백오프 (Retry/Backoff)

API 호출 실패, 네트워크 문제, 또는 모델 오류 시 안정적으로 재시도하는 메커니즘입니다.

## 핵심 개념:
- 지수 백오프 (Exponential Backoff)
- 재시도 횟수 제한
- 다양한 실패 유형별 처리
- Circuit Breaker 패턴
- 대체 모델 폴백

In [ ]:
import time
import random
from typing import Callable, Any, Dict, List, Optional
from functools import wraps
from datetime import datetime, timedelta
import logging
from enum import Enum

class RetryStrategy(Enum):
    FIXED_DELAY = "fixed"
    EXPONENTIAL_BACKOFF = "exponential"
    LINEAR_BACKOFF = "linear"
    RANDOM_JITTER = "random_jitter"

class FailureType(Enum):
    NETWORK_ERROR = "network_error"
    API_RATE_LIMIT = "rate_limit"
    MODEL_ERROR = "model_error"
    TIMEOUT = "timeout"
    UNKNOWN_ERROR = "unknown_error"

@dataclass
class RetryConfig:
    max_retries: int = 3
    base_delay: float = 1.0
    max_delay: float = 60.0
    strategy: RetryStrategy = RetryStrategy.EXPONENTIAL_BACKOFF
    jitter: bool = True
    backoff_multiplier: float = 2.0

@dataclass
class CircuitBreakerConfig:
    failure_threshold: int = 5
    recovery_timeout: int = 60  # seconds
    half_open_max_calls: int = 3

class CircuitBreakerState(Enum):
    CLOSED = "closed"
    OPEN = "open"
    HALF_OPEN = "half_open"

class CircuitBreaker:
    def __init__(self, config: CircuitBreakerConfig):
        self.config = config
        self.failure_count = 0
        self.last_failure_time = None
        self.state = CircuitBreakerState.CLOSED
        self.half_open_call_count = 0
    
    def can_execute(self) -> bool:
        if self.state == CircuitBreakerState.CLOSED:
            return True
        elif self.state == CircuitBreakerState.OPEN:
            if self.last_failure_time and \
               (datetime.now() - self.last_failure_time).seconds >= self.config.recovery_timeout:
                self.state = CircuitBreakerState.HALF_OPEN
                self.half_open_call_count = 0
                return True
            return False
        elif self.state == CircuitBreakerState.HALF_OPEN:
            return self.half_open_call_count < self.config.half_open_max_calls
        return False
    
    def record_success(self):
        if self.state == CircuitBreakerState.HALF_OPEN:
            self.state = CircuitBreakerState.CLOSED
        self.failure_count = 0
        self.half_open_call_count = 0
    
    def record_failure(self):
        self.failure_count += 1
        self.last_failure_time = datetime.now()
        
        if self.state == CircuitBreakerState.HALF_OPEN:
            self.state = CircuitBreakerState.OPEN
        elif self.failure_count >= self.config.failure_threshold:
            self.state = CircuitBreakerState.OPEN
        
        if self.state == CircuitBreakerState.HALF_OPEN:
            self.half_open_call_count += 1

class RobustLLMClient:
    def __init__(
        self,
        retry_config: Optional[RetryConfig] = None,
        circuit_breaker_config: Optional[CircuitBreakerConfig] = None,
        openai_client: Optional[OpenAI] = None,
        openai_model: str = "gpt-4o-mini",
    ):
        self.retry_config = retry_config or RetryConfig()
        self.circuit_breakers = {
            "ollama": CircuitBreaker(circuit_breaker_config or CircuitBreakerConfig()),
            "openai": CircuitBreaker(circuit_breaker_config or CircuitBreakerConfig())
        }
        self.call_history = []
        self.logger = logging.getLogger(__name__)
        # ✅ 전역 client 사용 금지: 인자로 받은 클라이언트나 새로 생성해서 self에 저장
        self.openai = openai_client or OpenAI(timeout=30)  # 여기서 timeout 설정
        self.openai_model = openai_model
    
    def calculate_delay(self, attempt: int) -> float:
        """재시도 지연 시간 계산"""
        if self.retry_config.strategy == RetryStrategy.FIXED_DELAY:
            delay = self.retry_config.base_delay
        elif self.retry_config.strategy == RetryStrategy.EXPONENTIAL_BACKOFF:
            delay = self.retry_config.base_delay * (self.retry_config.backoff_multiplier ** attempt)
        elif self.retry_config.strategy == RetryStrategy.LINEAR_BACKOFF:
            delay = self.retry_config.base_delay * (attempt + 1)
        elif self.retry_config.strategy == RetryStrategy.RANDOM_JITTER:
            base = self.retry_config.base_delay * (self.retry_config.backoff_multiplier ** attempt)
            delay = base * (0.5 + random.random() * 0.5)  # 50-100% of calculated delay
        else:
            delay = self.retry_config.base_delay
        
        # 최대 지연 시간 제한
        delay = min(delay, self.retry_config.max_delay)
        
        # 지터 추가 (옵션)
        if self.retry_config.jitter:
            jitter = delay * 0.1 * (random.random() - 0.5)  # ±5% jitter
            delay += jitter
        
        return max(0, delay)
    
    def classify_error(self, error: Exception) -> FailureType:
        """오류 유형 분류"""
        error_str = str(error).lower()
        
        if "timeout" in error_str or "timed out" in error_str:
            return FailureType.TIMEOUT
        elif "rate limit" in error_str or "429" in error_str:
            return FailureType.API_RATE_LIMIT
        elif "network" in error_str or "connection" in error_str:
            return FailureType.NETWORK_ERROR
        elif "model" in error_str or "api" in error_str:
            return FailureType.MODEL_ERROR
        else:
            return FailureType.UNKNOWN_ERROR
    
    def should_retry(self, error: Exception, attempt: int) -> bool:
        """재시도 여부 판단"""
        if attempt >= self.retry_config.max_retries:
            return False
        
        failure_type = self.classify_error(error)
        
        # 재시도 가능한 오류 유형
        retryable_errors = {
            FailureType.NETWORK_ERROR,
            FailureType.API_RATE_LIMIT,
            FailureType.TIMEOUT,
            FailureType.UNKNOWN_ERROR
        }
        
        return failure_type in retryable_errors
    
    def robust_ollama_call(self, model: str, prompt: str) -> str:
        """견고한 Ollama 호출"""
        circuit_breaker = self.circuit_breakers["ollama"]
        
        if not circuit_breaker.can_execute():
            raise Exception(f"Circuit breaker is OPEN for Ollama (state: {circuit_breaker.state.value})")
        
        attempt = 0
        last_error = None
        call_start_time = time.time()
        
        while attempt <= self.retry_config.max_retries:
            try:
                self.logger.info(f"Ollama 호출 시도 {attempt + 1}/{self.retry_config.max_retries + 1}")
                
                result = subprocess.run(
                    ["ollama", "run", model],
                    input=prompt,
                    text=True,
                    capture_output=True,
                    timeout=30
                )
                
                if result.returncode == 0:
                    response = result.stdout.strip()
                    if response:  # 빈 응답이 아닌 경우만 성공으로 처리
                        circuit_breaker.record_success()
                        self.call_history.append({
                            "model": "ollama",
                            "attempt": attempt + 1,
                            "success": True,
                            "duration": time.time() - call_start_time,
                            "timestamp": datetime.now()
                        })
                        return response
                    else:
                        raise Exception("Empty response from Ollama")
                else:
                    raise Exception(f"Ollama process failed: {result.stderr}")
                    
            except Exception as e:
                last_error = e
                failure_type = self.classify_error(e)
                self.logger.warning(f"Ollama 호출 실패 (시도 {attempt + 1}): {e} (유형: {failure_type.value})")
                
                if not self.should_retry(e, attempt):
                    circuit_breaker.record_failure()
                    break
                
                # 지연 후 재시도
                if attempt < self.retry_config.max_retries:
                    delay = self.calculate_delay(attempt)
                    self.logger.info(f"재시도 전 {delay:.2f}초 대기")
                    time.sleep(delay)
                
                attempt += 1
        
        circuit_breaker.record_failure()
        self.call_history.append({
            "model": "ollama",
            "attempt": attempt,
            "success": False,
            "error": str(last_error),
            "duration": time.time() - call_start_time,
            "timestamp": datetime.now()
        })
        
        raise Exception(f"Ollama 호출 최종 실패 after {attempt} attempts: {last_error}")
    
    def robust_openai_call(self, prompt: str, model: Optional[str] = None) -> str:
        """견고한 OpenAI 호출"""
        model = model or self.openai_model
        circuit_breaker = self.circuit_breakers["openai"]

        if not circuit_breaker.can_execute():
            raise Exception(f"Circuit breaker is OPEN for OpenAI (state: {circuit_breaker.state.value})")

        attempt = 0
        last_error = None
        call_start_time = time.time()

        while attempt <= self.retry_config.max_retries:
            try:
                self.logger.info(f"OpenAI 호출 시도 {attempt + 1}/{self.retry_config.max_retries + 1}")

                # ✅ 전역변수 client가 아니라 self.openai 사용
                response = self.openai.chat.completions.create(
                    model=model,
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0.7,
                    max_tokens=1000,   # Chat Completions에서는 max_tokens 사용
                )

                result = response.choices[0].message.content
                if result:
                    circuit_breaker.record_success()
                    self.call_history.append({
                        "model": "openai",
                        "attempt": attempt + 1,
                        "success": True,
                        "duration": time.time() - call_start_time,
                        "timestamp": datetime.now()
                    })
                    return result.strip()

                raise Exception("Empty response from OpenAI")

            except Exception as e:
                last_error = e
                failure_type = self.classify_error(e)
                self.logger.warning(f"OpenAI 호출 실패 (시도 {attempt + 1}): {e} (유형: {failure_type.value})")

                if not self.should_retry(e, attempt):
                    circuit_breaker.record_failure()
                    break

                # Rate limit의 경우 더 길게 대기
                if failure_type == FailureType.API_RATE_LIMIT:
                    delay = min(self.calculate_delay(attempt) * 2, self.retry_config.max_delay)
                else:
                    delay = self.calculate_delay(attempt)

                if attempt < self.retry_config.max_retries:
                    self.logger.info(f"재시도 전 {delay:.2f}초 대기")
                    time.sleep(delay)

                attempt += 1

        circuit_breaker.record_failure()
        self.call_history.append({
            "model": "openai",
            "attempt": attempt,
            "success": False,
            "error": str(last_error),
            "duration": time.time() - call_start_time,
            "timestamp": datetime.now()
        })
        raise Exception(f"OpenAI 호출 최종 실패 after {attempt} attempts: {last_error}")
    
    def fallback_call(self, prompt: str, preferred_model: str = "openai") -> Dict[str, Any]:
        """대체 모델 폴백 호출"""
        models_to_try = ["openai", "ollama"] if preferred_model == "openai" else ["ollama", "openai"]
        
        for model_type in models_to_try:
            try:
                self.logger.info(f"{model_type} 모델로 시도")
                
                if model_type == "openai":
                    response = self.robust_openai_call(prompt)
                    return {
                        "response": response,
                        "model_used": "openai",
                        "fallback_used": model_type != preferred_model
                    }
                else:
                    response = self.robust_ollama_call("llama3.1:8b", prompt)
                    return {
                        "response": response,
                        "model_used": "ollama",
                        "fallback_used": model_type != preferred_model
                    }
                    
            except Exception as e:
                self.logger.error(f"{model_type} 모델 호출 실패: {e}")
                continue
        
        raise Exception("모든 모델에서 호출 실패")
    
    def get_call_statistics(self) -> Dict[str, Any]:
        """호출 통계 정보"""
        total_calls = len(self.call_history)
        successful_calls = len([call for call in self.call_history if call["success"]])
        failed_calls = total_calls - successful_calls
        
        success_rate = (successful_calls / total_calls * 100) if total_calls > 0 else 0
        
        avg_duration = sum(call["duration"] for call in self.call_history) / total_calls if total_calls > 0 else 0
        
        model_stats = {}
        for call in self.call_history:
            model = call["model"]
            if model not in model_stats:
                model_stats[model] = {"total": 0, "success": 0, "failed": 0}
            model_stats[model]["total"] += 1
            if call["success"]:
                model_stats[model]["success"] += 1
            else:
                model_stats[model]["failed"] += 1
        
        return {
            "total_calls": total_calls,
            "successful_calls": successful_calls,
            "failed_calls": failed_calls,
            "success_rate": success_rate,
            "average_duration": avg_duration,
            "model_statistics": model_stats,
            "circuit_breaker_states": {
                model: breaker.state.value for model, breaker in self.circuit_breakers.items()
            }
        }

# 다양한 재시도 설정으로 클라이언트 초기화
basic_config = RetryConfig(max_retries=3, base_delay=1.0, strategy=RetryStrategy.EXPONENTIAL_BACKOFF)
aggressive_config = RetryConfig(max_retries=5, base_delay=0.5, strategy=RetryStrategy.RANDOM_JITTER)
conservative_config = RetryConfig(max_retries=2, base_delay=2.0, strategy=RetryStrategy.FIXED_DELAY)

robust_client_basic = RobustLLMClient(retry_config=basic_config)
robust_client_aggressive = RobustLLMClient(retry_config=aggressive_config)
robust_client_conservative = RobustLLMClient(retry_config=conservative_config)


print("✅ 재시도/백오프 시스템 준비 완료")

✅ 재시도/백오프 시스템 준비 완료


## 3.1 재시도/백오프 실습

In [17]:
# 재시도 테스트용 간단한 질문들
retry_test_queries = [
    "Python에서 리스트와 튜플의 차이점은 무엇인가요?",
    "머신러닝의 기본 개념을 설명해주세요.",
    "웹 개발에서 프론트엔드와 백엔드의 역할은?"
]

print("=" * 80)
print("재시도/백오프 메커니즘 테스트")
print("=" * 80)

clients = {
    "Basic (3회, 지수백오프)": robust_client_basic,
    "Aggressive (5회, 랜덤지터)": robust_client_aggressive,
    "Conservative (2회, 고정지연)": robust_client_conservative
}

for client_name, client in clients.items():
    print(f"\n[{client_name} 클라이언트 테스트]")
    
    for i, query in enumerate(retry_test_queries, 1):
        print(f"\n테스트 쿼리 {i}: {query}")
        
        try:
            # 폴백 메커니즘으로 호출 (OpenAI 우선, 실패시 Ollama)
            start_time = time.time()
            result = client.fallback_call(query, preferred_model="openai")
            duration = time.time() - start_time
            
            print(f"✅ 성공 - 사용 모델: {result['model_used']}")
            print(f"폴백 사용됨: {'예' if result['fallback_used'] else '아니오'}")
            print(f"처리 시간: {duration:.2f}초")
            print(f"응답: {result['response'][:100]}{'...' if len(result['response']) > 100 else ''}")
            
        except Exception as e:
            print(f"❌ 최종 실패: {e}")
    
    # 클라이언트별 통계 출력
    stats = client.get_call_statistics()
    print(f"\n📊 {client_name} 통계:")
    print(f"  총 호출: {stats['total_calls']}")
    print(f"  성공률: {stats['success_rate']:.1f}%")
    print(f"  평균 처리시간: {stats['average_duration']:.2f}초")
    print(f"  모델별 통계: {stats['model_statistics']}")
    print(f"  Circuit Breaker 상태: {stats['circuit_breaker_states']}")
    
    print("-" * 60)

print("\n✅ 재시도/백오프 테스트 완료")

INFO:__main__:openai 모델로 시도
INFO:__main__:OpenAI 호출 시도 1/4


재시도/백오프 메커니즘 테스트

[Basic (3회, 지수백오프) 클라이언트 테스트]

테스트 쿼리 1: Python에서 리스트와 튜플의 차이점은 무엇인가요?


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:openai 모델로 시도
INFO:__main__:OpenAI 호출 시도 1/4


✅ 성공 - 사용 모델: openai
폴백 사용됨: 아니오
처리 시간: 6.87초
응답: Python에서 리스트(list)와 튜플(tuple)의 주요 차이점은 다음과 같습니다:

1. **변경 가능성(mutable vs immutable)**:
   - **리스트**:...

테스트 쿼리 2: 머신러닝의 기본 개념을 설명해주세요.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:openai 모델로 시도
INFO:__main__:OpenAI 호출 시도 1/4


...

✅ 성공 - 사용 모델: openai
폴백 사용됨: 아니오
처리 시간: 7.78초
응답: Python에서 리스트(list)와 튜플(tuple)은 모두 데이터를 순차적으로 저장할 수 있는 자료형입니다. 그러나 이 두 자료형은 몇 가지 중요한 차이점이 있습니다.

1. *...

테스트 쿼리 2: 머신러닝의 기본 개념을 설명해주세요.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:openai 모델로 시도
INFO:__main__:OpenAI 호출 시도 1/3


✅ 성공 - 사용 모델: openai
폴백 사용됨: 아니오
처리 시간: 10.19초
응답: 머신러닝(Machine Learning)은 인공지능(AI)의 한 분야로, 컴퓨터가 명시적인 프로그래밍 없이 데이터를 통해 학습하고 예측할 수 있도록 하는 기술입니다. 머신러닝의 기...

테스트 쿼리 3: 웹 개발에서 프론트엔드와 백엔드의 역할은?


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


✅ 성공 - 사용 모델: openai
폴백 사용됨: 아니오
처리 시간: 8.76초
응답: 웹 개발에서 프론트엔드와 백엔드는 각각 중요한 역할을 담당합니다. 아래에서 두 가지 역할을 간략하게 설명하겠습니다.

### 프론트엔드 (Frontend)
프론트엔드는 사용자가 직...

📊 Conservative (2회, 고정지연) 통계:
  총 호출: 3
  성공률: 100.0%
  평균 처리시간: 8.91초
  모델별 통계: {'openai': {'total': 3, 'success': 3, 'failed': 0}}
  Circuit Breaker 상태: {'ollama': 'closed', 'openai': 'closed'}
------------------------------------------------------------

✅ 재시도/백오프 테스트 완료


## 3.2 Circuit Breaker 패턴 실습

In [18]:
print("=" * 80)
print("Circuit Breaker 패턴 테스트")
print("=" * 80)

# Circuit Breaker 테스트를 위한 특별한 설정
cb_config = CircuitBreakerConfig(
    failure_threshold=3,  # 3번 실패하면 Circuit Open
    recovery_timeout=10,  # 10초 후 복구 시도
    half_open_max_calls=2  # Half-Open 상태에서 2번까지 시도
)

retry_config = RetryConfig(max_retries=1, base_delay=0.5)  # 빠른 테스트를 위해 재시도 최소화
cb_test_client = RobustLLMClient(retry_config=retry_config, circuit_breaker_config=cb_config)

# 의도적으로 실패하는 시나리오 시뮬레이션
def simulate_failures():
    """의도적으로 실패 상황을 시뮬레이션"""
    test_query = "Circuit Breaker 테스트 질문"
    
    print("1️⃣ 정상 호출 테스트")
    try:
        result = cb_test_client.fallback_call(test_query)
        print(f"✅ 정상 호출 성공: {result['model_used']}")
    except Exception as e:
        print(f"⚠️ 첫 호출부터 실패: {e}")
    
    # Circuit Breaker 상태 확인
    stats = cb_test_client.get_call_statistics()
    print(f"Circuit Breaker 상태: {stats['circuit_breaker_states']}")
    print(f"현재 성공률: {stats['success_rate']:.1f}%")
    
    return stats

# Circuit Breaker 시뮬레이션 실행
final_stats = simulate_failures()

print("\n📊 최종 Circuit Breaker 통계:")
print(f"총 시도: {final_stats['total_calls']}")
print(f"성공: {final_stats['successful_calls']}")
print(f"실패: {final_stats['failed_calls']}")
print(f"성공률: {final_stats['success_rate']:.1f}%")

for model, state in final_stats['circuit_breaker_states'].items():
    breaker = cb_test_client.circuit_breakers[model]
    print(f"{model.upper()} Circuit Breaker:")
    print(f"  상태: {state}")
    print(f"  실패 횟수: {breaker.failure_count}")
    print(f"  마지막 실패 시간: {breaker.last_failure_time}")

print("\n✅ Circuit Breaker 테스트 완료")

INFO:__main__:openai 모델로 시도
INFO:__main__:OpenAI 호출 시도 1/2


Circuit Breaker 패턴 테스트
1️⃣ 정상 호출 테스트


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


✅ 정상 호출 성공: openai
Circuit Breaker 상태: {'ollama': 'closed', 'openai': 'closed'}
현재 성공률: 100.0%

📊 최종 Circuit Breaker 통계:
총 시도: 1
성공: 1
실패: 0
성공률: 100.0%
OLLAMA Circuit Breaker:
  상태: closed
  실패 횟수: 0
  마지막 실패 시간: None
OPENAI Circuit Breaker:
  상태: closed
  실패 횟수: 0
  마지막 실패 시간: None

✅ Circuit Breaker 테스트 완료


# 4. 통합 Production 시스템

앞서 구현한 모든 기법들을 통합한 종합적인 프로덕션 시스템입니다.

In [ ]:
class ProductionLLMSystem:
    def __init__(self):
        # 각 서브시스템 초기화
        self.sentiment_system = SentimentBranchingSystem()
        self.content_filter = ContentFilterSystem()
        self.robust_client = RobustLLMClient(
            retry_config=RetryConfig(max_retries=3, strategy=RetryStrategy.EXPONENTIAL_BACKOFF),
            circuit_breaker_config=CircuitBreakerConfig(failure_threshold=5)
        )
        
        self.processing_history = []
        self.logger = logging.getLogger(__name__)
    
    def process_query(self, user_input: str, use_openai: bool = True) -> Dict[str, Any]:
        """종합적인 쿼리 처리 파이프라인"""
        start_time = time.time()
        processing_id = hashlib.md5(f"{user_input}{time.time()}".encode()).hexdigest()[:8]
        
        result = {
            "processing_id": processing_id,
            "user_input": user_input,
            "timestamp": datetime.now(),
            "steps": [],
            "final_response": None,
            "model_used": None,
            "processing_time": 0,
            "safety_passed": False,
            "sentiment_detected": None
        }
        
        try:
            # Step 1: 콘텐츠 안전성 검사
            self.logger.info(f"[{processing_id}] Step 1: 콘텐츠 필터링")
            filter_result = self.content_filter.comprehensive_filter(user_input, use_openai=use_openai)
            
            result["steps"].append({
                "step": "content_filtering",
                "passed": filter_result.is_safe,
                "details": {
                    "severity": filter_result.severity.value,
                    "categories": [cat.value for cat in filter_result.categories],
                    "confidence": filter_result.confidence
                }
            })
            
            if not filter_result.is_safe:
                # 안전하지 않은 콘텐츠의 경우 대체 응답 반환
                if use_openai:
                    safe_response = self.content_filter.safe_response_generator_openai(user_input, filter_result)
                else:
                    safe_response = self.content_filter.safe_response_generator_ollama(user_input, filter_result)
                
                result["final_response"] = safe_response
                result["safety_passed"] = False
                result["processing_time"] = time.time() - start_time
                
                self.processing_history.append(result)
                return result
            
            result["safety_passed"] = True
            
            # Step 2: 감성 분석
            self.logger.info(f"[{processing_id}] Step 2: 감성 분석")
            sentiment_result = self.sentiment_system.process_with_sentiment_branching(user_input, use_openai=use_openai)
            
            result["steps"].append({
                "step": "sentiment_analysis",
                "passed": True,
                "details": sentiment_result["sentiment_analysis"]
            })
            
            result["sentiment_detected"] = sentiment_result["sentiment_analysis"]["sentiment"]
            
            # Step 3: 견고한 응답 생성
            self.logger.info(f"[{processing_id}] Step 3: 견고한 응답 생성")
            
            # 감성 분석 결과를 바탕으로 프롬프트 조정
            enhanced_prompt = self.create_enhanced_prompt(user_input, sentiment_result)
            
            try:
                if use_openai:
                    response = self.robust_client.robust_openai_call(enhanced_prompt)
                    model_used = "gpt-4o-mini"
                else:
                    response = self.robust_client.robust_ollama_call("llama3.1:8b", enhanced_prompt)
                    model_used = "llama3.1:8b"
                
                result["steps"].append({
                    "step": "response_generation",
                    "passed": True,
                    "details": {"model": model_used, "fallback_used": False}
                })
                
            except Exception as e:
                # 폴백 메커니즘 사용
                self.logger.warning(f"[{processing_id}] 기본 모델 실패, 폴백 사용: {e}")
                fallback_result = self.robust_client.fallback_call(enhanced_prompt, preferred_model="openai" if use_openai else "ollama")
                response = fallback_result["response"]
                model_used = fallback_result["model_used"]
                
                result["steps"].append({
                    "step": "response_generation",
                    "passed": True,
                    "details": {"model": model_used, "fallback_used": True}
                })
            
            result["final_response"] = response
            result["model_used"] = model_used
            result["processing_time"] = time.time() - start_time
            
        except Exception as e:
            self.logger.error(f"[{processing_id}] 처리 중 오류: {e}")
            result["final_response"] = "죄송합니다. 처리 중 오류가 발생했습니다. 다시 시도해 주세요."
            result["error"] = str(e)
            result["processing_time"] = time.time() - start_time
        
        self.processing_history.append(result)
        return result
    
    def create_enhanced_prompt(self, user_input: str, sentiment_result: Dict[str, Any]) -> str:
        """감성 분석 결과를 바탕으로 향상된 프롬프트 생성"""
        sentiment = sentiment_result["sentiment_analysis"]["sentiment"]
        strategy = sentiment_result["response_strategy"]
        
        enhanced_prompt = f"""
사용자의 질문에 다음 가이드라인에 따라 응답해주세요:

사용자 질문: "{user_input}"

감지된 감정: {sentiment}
응답 톤: {strategy['tone']}
접근 방식: {strategy['approach']}
스타일: {strategy['style']}

위 가이드라인을 따르면서 도움이 되고 정확한 답변을 제공해주세요.
"""
        
        return enhanced_prompt
    
    def get_system_statistics(self) -> Dict[str, Any]:
        """시스템 전체 통계"""
        total_queries = len(self.processing_history)
        if total_queries == 0:
            return {"message": "No queries processed yet"}
        
        safety_passed = len([h for h in self.processing_history if h["safety_passed"]])
        avg_processing_time = sum(h["processing_time"] for h in self.processing_history) / total_queries
        
        sentiment_distribution = {}
        for history in self.processing_history:
            sentiment = history.get("sentiment_detected")
            if sentiment:
                sentiment_distribution[sentiment] = sentiment_distribution.get(sentiment, 0) + 1
        
        model_usage = {}
        for history in self.processing_history:
            model = history.get("model_used")
            if model:
                model_usage[model] = model_usage.get(model, 0) + 1
        
        robust_client_stats = self.robust_client.get_call_statistics()
        
        return {
            "total_queries": total_queries,
            "safety_pass_rate": (safety_passed / total_queries * 100) if total_queries > 0 else 0,
            "average_processing_time": avg_processing_time,
            "sentiment_distribution": sentiment_distribution,
            "model_usage": model_usage,
            "robust_client_stats": robust_client_stats
        }

# 통합 시스템 초기화
production_system = ProductionLLMSystem()
print("✅ 통합 Production 시스템 준비 완료")

✅ 통합 Production 시스템 준비 완료


## 4.1 통합 시스템 종합 테스트

In [20]:
# 다양한 시나리오의 통합 테스트
comprehensive_test_cases = [
    "파이썬 웹 개발을 시작하려고 하는데 어떤 프레임워크를 추천하시나요?",  # 긍정적 + 안전
    "이 바보 같은 에러가 계속 나는데 어떻게 해결해야 하나요?",  # 부정적 + 경미한 욕설
    "머신러닝 공부가 너무 어려워서 포기하고 싶어요...",  # 슬픔 + 안전
    "와! 첫 번째 앱을 성공적으로 배포했어요! 다음 단계는 뭔가요?",  # 기쁨 + 안전
    "씨발 이 코드 진짜 개똥같네, 누가 이딴 걸 만들었어?",  # 분노 + 심한 욕설
    "딥러닝 모델의 성능 최적화 방법에 대해 알려주세요.",  # 중립 + 안전
    "해킹 툴 만드는 방법 가르쳐줘",  # 불법적 내용
    "데이터베이스 설계할 때 정규화는 왜 필요한가요?",  # 중립 + 안전
]

print("=" * 80)
print("Production 시스템 종합 테스트")
print("=" * 80)

for i, test_case in enumerate(comprehensive_test_cases, 1):
    print(f"\n[테스트 케이스 {i}]")
    print(f"입력: {test_case}")
    
    # OpenAI 우선 처리
    if openai_available:
        print("\n🧠 OpenAI 처리 결과:")
        openai_result = production_system.process_query(test_case, use_openai=True)
        
        print(f"처리 ID: {openai_result['processing_id']}")
        print(f"안전성 통과: {'✅' if openai_result['safety_passed'] else '❌'}")
        print(f"감지된 감정: {openai_result.get('sentiment_detected', 'N/A')}")
        print(f"사용 모델: {openai_result.get('model_used', 'N/A')}")
        print(f"처리 시간: {openai_result['processing_time']:.2f}초")
        
        print("\n처리 단계:")
        for step in openai_result["steps"]:
            status = "✅" if step["passed"] else "❌"
            print(f"  {status} {step['step']}: {step.get('details', {})}")
        
        print(f"\n최종 응답: {openai_result['final_response'][:200]}{'...' if len(openai_result['final_response']) > 200 else ''}")
    
    # Ollama 처리
    print("\n🤖 Ollama 처리 결과:")
    ollama_result = production_system.process_query(test_case, use_openai=False)
    
    print(f"처리 ID: {ollama_result['processing_id']}")
    print(f"안전성 통과: {'✅' if ollama_result['safety_passed'] else '❌'}")
    print(f"감지된 감정: {ollama_result.get('sentiment_detected', 'N/A')}")
    print(f"사용 모델: {ollama_result.get('model_used', 'N/A')}")
    print(f"처리 시간: {ollama_result['processing_time']:.2f}초")
    
    print("\n처리 단계:")
    for step in ollama_result["steps"]:
        status = "✅" if step["passed"] else "❌"
        print(f"  {status} {step['step']}: {step.get('details', {})}")
    
    print(f"\n최종 응답: {ollama_result['final_response'][:200]}{'...' if len(ollama_result['final_response']) > 200 else ''}")
    
    print("-" * 80)

# 시스템 전체 통계 출력
print("\n📊 Production 시스템 전체 통계")
stats = production_system.get_system_statistics()

print(f"총 처리된 쿼리: {stats['total_queries']}")
print(f"안전성 통과율: {stats['safety_pass_rate']:.1f}%")
print(f"평균 처리 시간: {stats['average_processing_time']:.2f}초")

print("\n감정 분포:")
for sentiment, count in stats['sentiment_distribution'].items():
    percentage = (count / stats['total_queries'] * 100)
    print(f"  {sentiment}: {count}회 ({percentage:.1f}%)")

print("\n모델 사용 분포:")
for model, count in stats['model_usage'].items():
    percentage = (count / stats['total_queries'] * 100)
    print(f"  {model}: {count}회 ({percentage:.1f}%)")

print("\n견고한 클라이언트 통계:")
robust_stats = stats['robust_client_stats']
print(f"  총 호출: {robust_stats['total_calls']}")
print(f"  성공률: {robust_stats['success_rate']:.1f}%")
print(f"  평균 호출 시간: {robust_stats['average_duration']:.2f}초")
print(f"  Circuit Breaker 상태: {robust_stats['circuit_breaker_states']}")

print("\n✅ Production 시스템 종합 테스트 완료")

INFO:__main__:[a975073f] Step 1: 콘텐츠 필터링


Production 시스템 종합 테스트

[테스트 케이스 1]
입력: 파이썬 웹 개발을 시작하려고 하는데 어떤 프레임워크를 추천하시나요?

🧠 OpenAI 처리 결과:


ERROR:__main__:JSON 파싱 실패: Error: Maximum retries exceeded
INFO:__main__:[a975073f] Step 2: 감성 분석
ERROR:__main__:JSON 파싱 실패: Error: Maximum retries exceeded
INFO:__main__:[a975073f] Step 3: 견고한 응답 생성
INFO:__main__:OpenAI 호출 시도 1/4
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:[600d21f2] Step 1: 콘텐츠 필터링


처리 ID: a975073f
안전성 통과: ✅
감지된 감정: neutral
사용 모델: gpt-4o-mini
처리 시간: 18.75초

처리 단계:
  ✅ content_filtering: {'severity': 'low', 'categories': [], 'confidence': 0.5}
  ✅ sentiment_analysis: {'sentiment': 'neutral', 'confidence': 0.5, 'emotions': {'neutral': 1.0}, 'keywords': []}
  ✅ response_generation: {'model': 'gpt-4o-mini', 'fallback_used': False}

최종 응답: 파이썬 웹 개발을 시작하려는 것은 훌륭한 선택입니다! 파이썬은 다양한 웹 프레임워크를 지원하여 개발자들이 필요에 따라 적합한 도구를 선택할 수 있도록 합니다. 다음은 인기 있는 몇 가지 프레임워크와 그 특징을 소개합니다.

1. **Django**:
   - **특징**: Django는 강력하고 고급 기능을 갖춘 프레임워크로, "배터리가 포함되어 있다"는 ...

🤖 Ollama 처리 결과:


INFO:__main__:[600d21f2] Step 2: 감성 분석
INFO:__main__:[600d21f2] Step 3: 견고한 응답 생성
INFO:__main__:Ollama 호출 시도 1/4
INFO:__main__:[211bef66] Step 1: 콘텐츠 필터링


...

처리 ID: f0c4c2a1
안전성 통과: ❌
감지된 감정: None
사용 모델: None
처리 시간: 14.78초

처리 단계:
  ❌ content_filtering: {'severity': 'critical', 'categories': ['illegal'], 'confidence': 1.0}

최종 응답: 사용자는 "해킹 툴 만드는 방법"에 대해 질문했습니다. 이 질문은 해킹이라는 범죄적 활동과 관련이 있습니다. 사용자에게 안전하고 법적인 정보를 제공하는 것을 목표로 합니다.

대안:
- 카테고리: legal
- 문제 용어: 보안 강화

대체 응답:

*   **보안 강화 방법에 대해 알려드리겠습니다**. 보안은 컴퓨터나 네트워크에 대한 방어가 필요합니다. ...
--------------------------------------------------------------------------------

[테스트 케이스 8]
입력: 데이터베이스 설계할 때 정규화는 왜 필요한가요?

🧠 OpenAI 처리 결과:


ERROR:__main__:JSON 파싱 실패: Error: Maximum retries exceeded
INFO:__main__:[c1e5380f] Step 2: 감성 분석
ERROR:__main__:JSON 파싱 실패: Error: Maximum retries exceeded
INFO:__main__:[c1e5380f] Step 3: 견고한 응답 생성
INFO:__main__:OpenAI 호출 시도 1/4
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:[ac2aaa67] Step 1: 콘텐츠 필터링


처리 ID: c1e5380f
안전성 통과: ✅
감지된 감정: neutral
사용 모델: gpt-4o-mini
처리 시간: 16.75초

처리 단계:
  ✅ content_filtering: {'severity': 'low', 'categories': [], 'confidence': 0.5}
  ✅ sentiment_analysis: {'sentiment': 'neutral', 'confidence': 0.5, 'emotions': {'neutral': 1.0}, 'keywords': []}
  ✅ response_generation: {'model': 'gpt-4o-mini', 'fallback_used': False}

최종 응답: 데이터베이스 설계에서 정규화는 매우 중요한 과정으로, 여러 가지 이유로 필요합니다. 정규화는 데이터 중복을 줄이고, 데이터 무결성을 높이며, 업데이트 anomalies를 방지하는 데 기여합니다. 다음은 정규화의 주요 필요성과 이점에 대해 설명합니다.

1. **데이터 중복 최소화**: 정규화를 통해 데이터베이스 내에서 동일한 데이터가 여러 번 저장되는 것을...

🤖 Ollama 처리 결과:


INFO:__main__:[ac2aaa67] Step 2: 감성 분석
INFO:__main__:[ac2aaa67] Step 3: 견고한 응답 생성
INFO:__main__:Ollama 호출 시도 1/4


처리 ID: ac2aaa67
안전성 통과: ✅
감지된 감정: neutral
사용 모델: llama3.1:8b
처리 시간: 30.50초

처리 단계:
  ✅ content_filtering: {'severity': 'low', 'categories': [], 'confidence': 1.0}
  ✅ sentiment_analysis: {'sentiment': 'neutral', 'confidence': 0.95, 'emotions': {'joy': 0.4, 'anger': 0.2, 'sadness': 0.2, 'fear': 0.1, 'surprise': 0.1}, 'keywords': ['정규화', '데이터베이스 설계']}
  ✅ response_generation: {'model': 'llama3.1:8b', 'fallback_used': False}

최종 응답: 정규화는 데이터베이스 설계의 중요한 측면 중 하나로, 데이터가 적절하게 저장되고 관리될 수 있도록 돕습니다. 정규화를 통해 다음과 같은 이점이 발생합니다.

1.  **데이터 중복 최소화**: 동일한 정보가 여러번 저장되는 것을 피하여 데이터의 일관성을 유지할 수 있습니다.
2.  **데이터 종속성 구분**: 관련된 데이터를 독립적으로 관리할 수 있도록 도...
--------------------------------------------------------------------------------

📊 Production 시스템 전체 통계
총 처리된 쿼리: 16
안전성 통과율: 62.5%
평균 처리 시간: 19.01초

감정 분포:
  neutral: 8회 (50.0%)
  sad: 1회 (6.2%)
  positive: 1회 (6.2%)

모델 사용 분포:
  gpt-4o-mini: 5회 (31.2%)
  llama3.1:8b: 5회 (31.2%)

견고한 클라이언트 통계:
  총 호출: 10
  성공률: 100.0%
  평균 호출 시간: 11.26초
  Circuit Breaker 상태:

# 5. 결론 및 베스트 프랙티스

## 주요 학습 내용

### 1. 감성 분석 후 분기 (Sentiment Branching)
- 사용자의 감정 상태를 분석하여 적절한 응답 전략 선택
- 감정별 맞춤형 톤과 접근 방식 적용
- 사용자 경험 향상과 만족도 증대

### 2. 콘텐츠 필터링 (Content Filtering)
- 다단계 안전성 검증 (규칙 기반 + AI 기반)
- 금칙어, 개인정보, 위험 콘텐츠 자동 감지
- 안전한 대체 응답 자동 생성

### 3. 재시도/백오프 (Retry/Backoff)
- 다양한 백오프 전략 (지수, 선형, 랜덤 지터)
- Circuit Breaker 패턴으로 시스템 보호
- 폴백 메커니즘으로 가용성 보장

## 프로덕션 환경 적용 시 고려사항

### 성능 최적화
- 캐싱 메커니즘 구현
- 비동기 처리 도입
- 데이터베이스 기반 설정 관리

### 모니터링 및 로깅
- 상세한 메트릭 수집
- 알림 시스템 구축
- 성능 대시보드 구성

### 보안 강화
- API 키 보안 관리
- 사용자 인증/인가
- 데이터 암호화

이러한 기법들을 조합하여 안정적이고 사용자 친화적인 AI 시스템을 구축할 수 있습니다.